In [0]:
from google.colab import files, auth, drive

In [2]:
# Mount to drive
drive.mount('/content/gdrive', force_remount=True)
data_dir_drive ='/content/gdrive/My Drive/Colab Notebooks/NLP1/Lab3'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
! pip install nltk

In [7]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

It is a type of chart parsing (in Romanian: parser cu agenda). It is used in order to avoi rebuilding subtrees that are already correct. All the above parsers used backtracking in their algorithm, therefore, we could have subtrees that are processed again because there is a wrong assumption in the subtrees created before them. It's in backtracking's nature to delete(forget) all the computing it made for a wrong prefix in the sollution. Each time we build a subtree we save it in a "table" and it will be reused when it's needed. An idea of implementation is given in the NLTK book (Natural Language Processing with Python, by Steven Bird, Ewan Klein, and Edward Loper, 2009).

1. Supposing we have a sentence of n words, we create a matrix of (n+1)2elements. Let's call this matrix T. The meaning of this matrix is that T[i][j] wiil contan the root of the subtree containing all the words from i to j-1. At first, the matrix will be empty (initialisez with a null value), except elements T[i][i+1] that will contain the i-th word.
2. Next we continuosly apply the productions completing the table, until no more changes in the table are made. In order to complete T[i][j] with a label, we must have a number k in [0,n] such that T[i][k] and T[k][j] are both completed (let's say T[i][k] is B and T[k][j] is C) and a production A -< B C. In this case, we'll assign T[i][j]=A. We may have multiple cases for the same line i and column j (from a different reduction of the trees). In this case, we save all the values, so it is better to consider T[i][j] being a list of symbols. an even better representantion, in order to easily obtain the responsible productions for the end tree, would be to have the whole production (A -< B C) saved in T[i][j], for example by saving it's id (assuming that all the grammar's productions have an id).
3. How do we treat productions with a number of terminals not equal to 2. We may process the grammar and reform the productions, by adding auxiliary ones in order to obtain only two node in each production; for example "A-> B C D" can be changed into "A-> B NewT1" and "NewT1-> C D", where NewT1 is a new terminal used only for this production.
4. The algorithm finishes when no more reductions can be made. If we've obtained S (the sentence node) in T[0][n] we have succesfully parsed the sentence.

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

In [0]:
from nltk import CFG, Nonterminal, Production
grammar = nltk.CFG.fromstring("""  S -> NP VP | TO VB
                                   VP -> V NP | V NP PP | V S | V PP
                                   PP -> P NP  
                                   V -> "caught" | "ate" | "likes" | "like" | "chase" | "go"
                                   NP -> Det N | Det N PP | PRP
                                   Det -> "the" | "a" | "an" | "my" | "some"
                                   N -> "mice" | "cat" | "dog" |  "school"
                                   P -> "in" | "to" | "on"
                                   TO -> "to"
                                   PRP -> "I"  """)

In [85]:
#step 3
import queue

productions = grammar.productions()
productions_ok = [] # <3 nonterminals
productions_not_ok = queue.Queue() # >2 nonterminals

for production in productions:
    if len(production.rhs()) <= 2:
        productions_ok.append(production)
    else:
        productions_not_ok.put(production)
        print(production)

print()
index = 1
while not productions_not_ok.empty():
    production = productions_not_ok.get()
    if len(production.rhs()) <= 2: #is ok
        productions_ok.append(production)
        print(production)
    else:
        # create a new productionsȘ "A-> B C D..." can be changed into "A-> B NewT1" and "NewT1-> C D...", 
        productions_ok.append(Production(production.lhs(), [production.rhs()[0], Nonterminal('Artificial' + str(index))])) # "A-> B NewT1" is ok
        print(Production(production.lhs(), [production.rhs()[0], Nonterminal('Artificial' + str(index))]))
        productions_not_ok.put(Production(Nonterminal('Artificial' + str(index)), production.rhs()[1:])) # "NewT1-> C D..." i dont know if is ok
        index += 1

print(productions_ok)
grammar = CFG(grammar.start(), productions_ok)
print(grammar)

VP -> V NP PP
NP -> Det N PP

VP -> V Artificial1
NP -> Det Artificial2
Artificial1 -> NP PP
Artificial2 -> N PP
[S -> NP VP, S -> TO VB, VP -> V NP, VP -> V S, VP -> V PP, PP -> P NP, V -> 'caught', V -> 'ate', V -> 'likes', V -> 'like', V -> 'chase', V -> 'go', NP -> Det N, NP -> PRP, Det -> 'the', Det -> 'a', Det -> 'an', Det -> 'my', Det -> 'some', N -> 'mice', N -> 'cat', N -> 'dog', N -> 'school', P -> 'in', P -> 'to', P -> 'on', TO -> 'to', PRP -> 'I', VP -> V Artificial1, NP -> Det Artificial2, Artificial1 -> NP PP, Artificial2 -> N PP]
Grammar with 32 productions (start state = S)
    S -> NP VP
    S -> TO VB
    VP -> V NP
    VP -> V S
    VP -> V PP
    PP -> P NP
    V -> 'caught'
    V -> 'ate'
    V -> 'likes'
    V -> 'like'
    V -> 'chase'
    V -> 'go'
    NP -> Det N
    NP -> PRP
    Det -> 'the'
    Det -> 'a'
    Det -> 'an'
    Det -> 'my'
    Det -> 'some'
    N -> 'mice'
    N -> 'cat'
    N -> 'dog'
    N -> 'school'
    P -> 'in'
    P -> 'to'
    P -> 'on'

In [86]:
productions = grammar.productions()
productions_ok = [] # A -> C D
productions_not_ok = [] # A -> B

for production in productions:
    if len(production.rhs()) == 2:
        productions_ok.append(production)
    else:
        if type(production.rhs()[0]) is Nonterminal:
            productions_not_ok.append(production)
            production_aux = grammar.productions(lhs=production.rhs()[0])
            print(production_aux)
            for prod_aux in production_aux:
                productions_ok.append(Production(production.lhs(), prod_aux.rhs()))
                print(Production(production.lhs(), prod_aux.rhs()))
        else:
            productions_ok.append(production)

print(productions_not_ok)
grammar = CFG(grammar.start(), productions_ok + productions_not_ok)
print(grammar)

[PRP -> 'I']
NP -> 'I'
[NP -> PRP]
Grammar with 33 productions (start state = S)
    S -> NP VP
    S -> TO VB
    VP -> V NP
    VP -> V S
    VP -> V PP
    PP -> P NP
    V -> 'caught'
    V -> 'ate'
    V -> 'likes'
    V -> 'like'
    V -> 'chase'
    V -> 'go'
    NP -> Det N
    NP -> 'I'
    Det -> 'the'
    Det -> 'a'
    Det -> 'an'
    Det -> 'my'
    Det -> 'some'
    N -> 'mice'
    N -> 'cat'
    N -> 'dog'
    N -> 'school'
    P -> 'in'
    P -> 'to'
    P -> 'on'
    TO -> 'to'
    PRP -> 'I'
    VP -> V Artificial1
    NP -> Det Artificial2
    Artificial1 -> NP PP
    Artificial2 -> N PP
    NP -> PRP


In [96]:
#step 1 and 2 and 4
def well_formed_substrings_table(sentence = "", grammar = grammar):
    # 1
    words = word_tokenize(sentence)
    words2 = [x for x in words if x.isalnum()]
    n = len(words2)
    T = np.empty([n + 1, n + 1], dtype='object')
    for i in range(n + 1):
        for j in range(n + 1):
            T[i][j] = set()

    for i, word in enumerate(words2):
        productions = grammar.productions(rhs = word)
        for production in productions:
            if len(production.rhs()) == 1:
                T[i][i+1].add(production)

    # 2
    while True:
        change = False
        for i in range(n + 1):
            for k in range(n + 1):
                for j in range(i + 1, n + 1):
                    productions1 = T[i][k]
                    productions2 = T[k][j]
                    for production1 in productions1:
                        for production2 in productions2:
                            for production in grammar.productions():
                                if len(production.rhs()) == 2:
                                    if production.rhs()[0] == production1.lhs() and production.rhs()[1] == production2.lhs():
                                        if production not in T[i][j]:
                                            T[i][j].add(production)
                                            change = True
        if not change:
            break
    print(T)
    # 4
    for production in T[0][n]:
        if production.lhs() == grammar.start():
            return True
    return False

T = well_formed_substrings_table('I like my dog', grammar)
print(T)
print(grammar)

[[set() {PRP -> 'I', NP -> 'I'} set() set() {S -> NP VP}]
 [set() set() {V -> 'like'} set() {VP -> V NP}]
 [set() set() set() {Det -> 'my'} {NP -> Det N}]
 [set() set() set() set() {N -> 'dog'}]
 [set() set() set() set() set()]]
True
Grammar with 33 productions (start state = S)
    S -> NP VP
    S -> TO VB
    VP -> V NP
    VP -> V S
    VP -> V PP
    PP -> P NP
    V -> 'caught'
    V -> 'ate'
    V -> 'likes'
    V -> 'like'
    V -> 'chase'
    V -> 'go'
    NP -> Det N
    NP -> 'I'
    Det -> 'the'
    Det -> 'a'
    Det -> 'an'
    Det -> 'my'
    Det -> 'some'
    N -> 'mice'
    N -> 'cat'
    N -> 'dog'
    N -> 'school'
    P -> 'in'
    P -> 'to'
    P -> 'on'
    TO -> 'to'
    PRP -> 'I'
    VP -> V Artificial1
    NP -> Det Artificial2
    Artificial1 -> NP PP
    Artificial2 -> N PP
    NP -> PRP
